In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import holoviews as hv
from holoviews import opts
hv.extension('bokeh')
import altair as alt
import seaborn as sns
from matplotlib import pyplot as plt


from fuzzywuzzy import fuzz 
from fuzzywuzzy import process 

#Apriori libraries 
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

import nltk

import warnings
warnings.filterwarnings("ignore")

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Import Dataset and check the rows and columns

In [ ]:
df = pd.read_csv("../input/gufhtugu-publications-dataset-challenge/GP Orders - 5.csv", delimiter=',')
df.head()

# Shape of dataset <br/>
We have 19187 rows and 5 columns

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
df.info()

# Rename the column of dataset for easy

In [ ]:
df = df.rename(columns={'Order Number': 'order_number',"Order Status":"order_status", "Book Name":"book_name","Order Date & Time":"order_date","City":"billing_city", "Payment Method":"payment_method", "Total items":"total_books", "Total weight (grams)":"grams"})
df.head()

# Check missing value in dataset

In [ ]:
df.isnull().sum().sort_values(ascending = False)

**Check NaN in book_name**

In [ ]:
df[df['book_name'].isna()]

**Check NaN in billing_city**

In [ ]:
df[df['billing_city'].isna()]

Delete all NaN from dataframe

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.isnull().sum()

Now, We have no NaN values.

**We have 3 order status**

In [ ]:
df.order_status.unique()

In [ ]:
df.order_status.value_counts()

In [ ]:
df.order_status.hist()

# Data Cleaning

**Split the book_name column with '/'and make a new row**

In [ ]:
split_col = df['book_name'].str.split('/', expand=True).stack()

# Melting dataframe so that we have one book in each row
split_col.index = split_col.index.droplevel(-1) # to line up with df's index
split_col.name = 'book_name' # needs a name to join

df = df.drop(columns='book_name').join(split_col)
df.head(10)

**Make book_name and billing_city to lowercase**

In [ ]:
df['book_name'] = df['book_name'].str.lower()
df['billing_city'] = df['billing_city'].str.lower()

In [ ]:
#pd.set_option('display.max_rows', df.shape[0]+1)
df['billing_city'].head(10)

**We have 3518  total cities**

In [ ]:
df['billing_city'].nunique()

**Try to clean the cities column**

In [ ]:
# df['billing_city'] = df['billing_city'].replace({'KHI':'KARACHI', 'KHI':'KARACHI'})
# df['billing_city'] = df['billing_city'].replace({'GAGOO MANDI TEHSIL BUREWALA DISTRICT VEHARI':'VEHARI'})
# df['billing_city'] = df['billing_city'].replace({'ZILA TOBA TAK SING TAHSEEL GOJJRA':'GOJRA'})
# df['billing_city'] = df['billing_city'].replace({'WAH CANTT, TAXILA, RAWALPINDI':'RAWALPINDI'})
# df['billing_city'] = df['billing_city'].replace({'MIDEL TOWN LAHORE':'LAHORE'})
# df['billing_city'] = df['billing_city'].replace({'TEHSIL BARNALA, DUST. BHIMBER':'BHIMBER'})
# df['billing_city'] = df['billing_city'].replace({'ADDA ZAKHEERA(DUNYA PUR)':'DUNYAPUR'})
# df['billing_city'] = df['billing_city'].replace({'JAUHARABAD , DISTT KHUSHAB':'KHUSHAB'})
# df['billing_city'] = df['billing_city'].replace({'ADIYALA ROAD, RAWALPINDI':'RAWALPINDI'})
# df['billing_city'] = df['billing_city'].replace({'UPPER CHITRAL':'CHITRAL'})
# df['billing_city'] = df['billing_city'].replace({'MUSTUFA TOWN, WAHDAT ROAD, LAHORE.':'LAHORE'})
# df['billing_city'] = df['billing_city'].replace({'AHMED PUR SIAL JHANG':'JHANG'})
# df['billing_city'] = df['billing_city'].replace({'KALLAR SYEDAN DIST RAWALPINDI':'RAWALPINDI'})
# df['billing_city'] = df['billing_city'].replace({'BAHWALNAGAR':'BAHAWALNAGAR'})
# df['billing_city'] = df['billing_city'].replace({'LAHORE, PAKISTAN':'LAHORE'})
# df['billing_city'] = df['billing_city'].replace({'MADINA TOWN, FAISALABAD':'FAISALABAD'})
# df['billing_city'] = df['billing_city'].replace({'ALI RAZA ABAD 5KM RAIWIND ROAD LAHORE':'LAHORE'})
# df['billing_city'] = df['billing_city'].replace({'MUZAFFARABAD AZAD KASHMIR':'MUZAFFARABAD'})
# df['billing_city'] = df['billing_city'].replace({'TALUKA HALA DISTRICT MATIARI':'MATIARI'})
# df['billing_city'] = df['billing_city'].replace({'AGRICS TOWN, RAIWAND ROAD LAHORE':'LAHORE'})
# df['billing_city'] = df['billing_city'].replace({'LYARI, KARACHI':'KARACHI'})
# df['billing_city'] = df['billing_city'].replace({'NORTH NAZIMABAD KARACHI':'KARACHI'})
# df['billing_city'] = df['billing_city'].replace({'SAADI TOWN/KARACHI':'KARACHI'})
# df['billing_city'] = df['billing_city'].replace({'KORANGI, KARACHI':'KARACHI'})
# df['billing_city'] = df['billing_city'].replace({'RAWALPINDI CANT':'RAWALPINDI'})
# df['billing_city'] = df['billing_city'].replace({'GULSHAN E IQBAL BLOCK 10 A, KARACHI':'KARACHI'})
# df['billing_city'] = df['billing_city'].replace({'MALIR, KARACHI':'KARACHI'})
# df['billing_city'] = df['billing_city'].replace({'SABZAZAR, MULTAN ROAD,LAHORE':'LAHORE'})
# df['billing_city'] = df['billing_city'].replace({'TOWNSHIP LAHORE':'LAHORE'})
# df['billing_city'] = df['billing_city'].replace({'CHAKLALA RAWALPINDI':'RAWALPINDI'})
# df['billing_city'] = df['billing_city'].replace({'TEHSIL PHALIA DISTRICT MANDI BHUDDIN':'MANDI BAHAUDDIN'})
# df['billing_city'] = df['billing_city'].replace({'VILLAGE SHUKHDARA TEHSIL MATTA DISTRICT SWAT':'SWAT'})
# df['billing_city'] = df['billing_city'].replace({'DHA,EME SOCIETY MULTAN ROAD LAHORE PAKISTAN.':'LAHORE'})
# df['billing_city'] = df['billing_city'].replace({'PAHAR PUR D I KHAN':'D I KHAN'})
# df['billing_city'] = df['billing_city'].replace({'MANDI BAHAHUDIN':'MANDI BAHAUDDIN'})
# df['billing_city'] = df['billing_city'].replace({'FAZAL COLONY NEAR EMCO FACTORY SHEIKHUPURA ROA':'SHEIKHUPURA'})
# df['billing_city'] = df['billing_city'].replace({'SADIQ ABAD DISTRICT RAHIM YAR KHAN':'RAHIM YAR KHAN'})
# df['billing_city'] = df['billing_city'].replace({'CHOUPERHATTA KABIRWALA KHANEWAL':'KHANEWAL'})
# df['billing_city'] = df['billing_city'].replace({'ACADEMY TOWN, PESHAWAR':'PESHAWAR'})
# df['billing_city'] = df['billing_city'].replace({'LATIFABAD, HYDERABAD':'HYDERABAD'})
# df['billing_city'] = df['billing_city'].replace({'G-8/1, ISLAMABAD':'ISLAMABAD'})
# df['billing_city'] = df['billing_city'].replace({'ORANGI TOWN KARACHI':'KARACHI'})
# df['billing_city'] = df['billing_city'].replace({'CHASHMA, KUNDIAN, DISTRICT, MIANWALI':'MIANWALI'})
# df['billing_city'] = df['billing_city'].replace({'MALAKWAL DISTRICT MANDI BAHAUDDIN':'MANDI BAHAUDDIN'})
# df['billing_city'] = df['billing_city'].replace({'GULISTAN E JAUHAR KARACHI':'KARACHI'})
# df['billing_city'] = df['billing_city'].replace({'JABOKA OKARA':'OKARA'})
# df['billing_city'] = df['billing_city'].replace({'GHOURI TOWN, ISLAMABAD':'ISLAMABAD'})
# df['billing_city'] = df['billing_city'].replace({'GULZAR E HIJRI':'KARACHI'})
# df['billing_city'] = df['billing_city'].replace({'GULBERG TOWN KARACHI CENTRAL':'KARACHI'})
# df['billing_city'] = df['billing_city'].replace({'KAHNA NAU LAHORE':'LAHORE'})
# df['billing_city'] = df['billing_city'].replace({'FAZAL COLONY NEAR EMCO FACTORY SHEIKHUPURA ROAD LAHORE':'LAHORE'})
# df['billing_city'] = df['billing_city'].replace({'ISLAAM ABAD':'ISLAMABAD'})
# df['billing_city'] = df['billing_city'].replace({'TEHSIL ALLAI DISST BATTAGRAM':'BATTAGRAM'})
# df['billing_city'] = df['billing_city'].replace({'SWABI PESHAWAR':'SWABI'})
# df['billing_city'] = df['billing_city'].replace({'FEROZ PUR ROAD LAHORE':'LAHORE'})
# df['billing_city'] = df['billing_city'].replace({'SAKRAND DIST:NAWABSHAH':'NAWABSHAH'})
# df['billing_city'] = df['billing_city'].replace({'SARGODHA SILANWALI ROAD HAMEED TOWN':'SARGODHA'})
# df['billing_city'] = df['billing_city'].replace({'BAGH AZAD KASHMIR':'BAGH'})
# df['billing_city'] = df['billing_city'].replace({'KAMEER DARBAR HAZRAT BABA MUHAMMAD PANHA DISTRICT SAHIWAL':'SAHIWAL'})
# df['billing_city'] = df['billing_city'].replace({'RAWAL PINDI':'RAWALPINDI'})
# df['billing_city'] = df['billing_city'].replace({'FAISAL ABAD':'FAISALABAD'})
# df['billing_city'] = df['billing_city'].replace({'SIALKOT PASRUR':'SIALKOT'})
# df['billing_city'] = df['billing_city'].replace({'PESHAWAR GULBAHAR NO.1':'PESHAWAR'})
# df['billing_city'] = df['billing_city'].replace({'TAXILA, RAWALPINDI':'RAWALPINDI'})
# df['billing_city'] = df['billing_city'].replace({'NAWAB SHAH':'NAWABSHAH'})
# df['billing_city'] = df['billing_city'].replace({'CHAKDARA LOWER DIR':'DIR'})
# df['billing_city'] = df['billing_city'].replace({'LALIAN DISTRICT CHINIOT':'CHINIOT'})
# df['billing_city'] = df['billing_city'].replace({'BAHWALPUR':'BAHAWALPUR'})
# df['billing_city'] = df['billing_city'].replace({'DASKA, SIALKOT':'SIALKOT'})
# df['billing_city'] = df['billing_city'].replace({'UPPER DIR':'DIR'})
# df['billing_city'] = df['billing_city'].replace({'MAKHAI TIMERGATA':'TIMERGATA'})
# df['billing_city'] = df['billing_city'].replace({'HAZARA TOWN QUETTA':'QUETTA'})
# df['billing_city'] = df['billing_city'].replace({'QILA DIDAR SINGH, GUJRANWALA':'GUJRANWALA'})
# df['billing_city'] = df['billing_city'].replace({'F-17/2 ISLAMABAD':'ISLAMABAD'})
# df['billing_city'] = df['billing_city'].replace({'ALKHIDMAT RAAZI HOSPITAL CBR TOWN':'ISLAMABAD'})
# df['billing_city'] = df['billing_city'].replace({'ZAFARWAL ZILLA NAROWAL':'NAROWAL'})
# df['billing_city'] = df['billing_city'].replace({'DINGA, TEHSIL KHARIAN,DISTRICT GUJRAT.':'GUJRAT'})
# df['billing_city'] = df['billing_city'].replace({'BALDIA TOWN':'KARACHI'})
# df['billing_city'] = df['billing_city'].replace({'TEH: DASKA/ DISTT: SIALKOT':'SIALKOT'})
# df['billing_city'] = df['billing_city'].replace({'MUZAFFARGARH PAKISTAN':'MUZAFFARGARH'})
# df['billing_city'] = df['billing_city'].replace({'JOHAR TOWN, LAHORE':'LAHORE'})
# df['billing_city'] = df['billing_city'].replace({'B?GH AZAD KASHMIR':'BAGH'})
# df['billing_city'] = df['billing_city'].replace({'ARIFWALA DISTRICT PAKPATTAN':'ARIFWALA'})
# df['billing_city'] = df['billing_city'].replace({'LYYAH':'LAYYAH'})
# df['billing_city'] = df['billing_city'].replace({'M. B DIN':'MANDI BAHAUDDIN'})
# df['billing_city'] = df['billing_city'].replace({'MULTAN/LODHRAN':'MULTAN'})
# df['billing_city'] = df['billing_city'].replace({'MIRPUR AZAD KASHMIR':'MIRPUR'})
# df['billing_city'] = df['billing_city'].replace({'FATEH PUR DISTRICT LAYYAH':'LAYYAH'})
# df['billing_city'] = df['billing_city'].replace({'PESHAWAR RAHATABAD':'PESHAWAR'})
# df['billing_city'] = df['billing_city'].replace({'USTA MUHAMMAD DISTRICT JAFFERABAD':'USTA MUHAMMAD'})
# df['billing_city'] = df['billing_city'].replace({'BERON YAKATOOT ZARGAR ABAD PESHAWAR CITY':'PESHAWAR'})
# df['billing_city'] = df['billing_city'].replace({'KOTLA JAM DISTRICT BHAKKAR':'BHAKKAR'})
# df['billing_city'] = df['billing_city'].replace({'USMAN WALA DISTRICT KASUR PUNJAB PAKISTAN':'KASUR'})
# df['billing_city'] = df['billing_city'].replace({'BALAKOT DISTRICT MANSEHRA':'MANSEHRA'})
# df['billing_city'] = df['billing_city'].replace({'AHMAD PUR SIAL/JHANG':'JHANG'})
# df['billing_city'] = df['billing_city'].replace({'KARACHI/MALIR/QUAIDABAD':'KARACHI'})
# df['billing_city'] = df['billing_city'].replace({'USLAMABAF':'ISLAMABAD'})
# df['billing_city'] = df['billing_city'].replace({'BHIMBER AZAD KASHMIR':'BHIMBER'})
# df['billing_city'] = df['billing_city'].replace({'SITA ROAD RAHMANI NAGHAR DISTRICT DADU':'DADU'})
# df['billing_city'] = df['billing_city'].replace({'MANGAT MANDI BAHA UD DIN':'MANDI BAHAUDDIN'})
# df['billing_city'] = df['billing_city'].replace({'TEH:DASKA/ DISTT: SIALKOT':'SIALKOT'})
# df['billing_city'] = df['billing_city'].replace({'PINDI GHEB DISTRICT ATTOCK':'ATTOCK'})

In [ ]:
# @arindam et al.

# Source: https://simplemaps.com/data/pk-cities

pakistan_top_cities = ['karachi', 'lahore', 'sialkot', 'faisalabad', 'rawalpindi',
       'peshawar', 'saidu sharif', 'multan', 'gujranwala', 'islamabad',
       'quetta', 'bahawalpur', 'sargodha', 'new mirpur', 'chiniot',
       'sukkur', 'larkana', 'shekhupura', 'jhang', 'rahimyar khan',
       'gujrat', 'kasur', 'mardan', 'mingaora', 'dera ghazi khan',"dgk"
       'nawabshah', 'sahiwal', 'mirpur khas', 'okara', 'burewala',
       'jacobabad', 'saddiqabad', 'kohat', 'muridke', 'muzaffargarh',
       'khanpur', 'gojra', 'bahauddin', 'abbottabad', 'dadu',
       'khuzdar', 'pakpattan', 'tando allahyar', 'vihari', 'jaranwala',
       'kamalia', 'kot addu', 'nowshera', 'swabi', 'dera ismail khan',
       'chaman', 'charsadda', 'kandhkot', 'hasilpur', 'muzaffarabad',
       'mianwali', 'jalalpur","jattan', 'bhakkar', 'zhob', 'kharian',
       'mian channun', 'jamshoro', 'pattoki', 'harunabad',
       'toba tek singh', 'shakargarh', 'hujra", "shah", "muqim', 'kabirwala',
       'mansehra', 'lala musa', 'nankana sahib', 'bannu', 'timargara',
       'parachinar', 'gwadar', 'abdul hakim', 'hassan", "abdal', 'tank',
       'hangu', 'risalpur cantonment', 'karak', 'kundian', 'umarkot',
       'chitral', 'dainyor', 'kulachi', 'kotli', 'gilgit',
       'hyderabad', 'narowal', 'khairpur', "mir’s", 'khanewal', 'jhelum',
       'haripur', 'shikarpur', 'rawala kot', 'hafizabad', 'lodhran',
       'malakand', 'attock', 'batgram', 'matiari', 'ghotki',
       'firoz','naushahro', 'alpurai', 'bagh', 'daggar', 'bahawalnagar',
       'leiah', 'tando muhammad khan', 'chakwal', 'khushab', 'badin',
       'lakki', 'rajanpur', 'dera allahyar', 'shahdad kot', 'pishin',
       'sanghar', 'upper dir', 'thatta', 'dera murad jamali', 'kohlu',
       'mastung', 'dasu', 'athmuqam', 'loralai', 'barkhan',
       'musa khel bazar', 'ziarat', 'gandava', 'sibi', 'dera bugti',
       'eidgah', 'turbat', 'uthal', 'chilas', 'kalat', 'panjgur', 'gakuch',
       'qila', 'saifullah', 'kharan', 'aliabad', 'awaran', 'dalbandin']



In [ ]:
single_word_cities = df[df["billing_city"].str.split().apply(len) == 2]["billing_city"].unique()
single_word_cities[:20]

In [ ]:
def clean_city(row):
    address = row.billing_city.split()
    add = set()
    for a in address:
        a = a.strip()
        if a:
            add.add(a)
    for city in pakistan_top_cities:
        if row.billing_city.__contains__(city):
            return city
        
    for a in add:
        for c in pakistan_top_cities:
            if nltk.edit_distance(a, c) <= 5: # considering spelling mistakes upto 5 letters
                return c
    return row.billing_city

In [ ]:
# Number of cities before cleaning
df["billing_city"].nunique()

In [ ]:
df["billing_city"] = df.apply(clean_city, axis=1)

In [ ]:
df['billing_city'].head()

In [ ]:
df['billing_city'].nunique()

In [ ]:
df.head()

# Convert the 'order_date' column to datetime format

In [ ]:
# df['order_date']= pd.to_datetime(df['order_date'])
 
# #Extracting year,month and day
# df['year'] = df['order_date'].apply(lambda x : x.year)
# df['month'] = df['order_date'].apply(lambda x : x.month_name)
# df['day'] = df['order_date'].apply(lambda x : x.day)
# df['weekday'] = df['order_date'].apply(lambda x : x.weekday())

# #Rearranging the columns
# df_new=df[['order_number', 'order_status', 'book_name', 'order_date', 'billing_city', 'year', 'month', 'day','weekday']]
# df_new.head()

# Convert the 'day' column to day_name

In [ ]:

df['order_date']= pd.to_datetime(df['order_date'])
 
#Extracting year,month and day
df['year'] = df['order_date'].apply(lambda x : x.year)
df['month'] = df['order_date'].apply(lambda x : x.month)
df['day'] = df['order_date'].apply(lambda x : x.day_name())
df['weekday'] = df['order_date'].apply(lambda x : x.weekday())

#Rearranging the columns
df_new=df[['order_number', 'order_status', 'book_name', 'order_date', 'billing_city', 'year', 'month', 'day','weekday']]
df_new.head()

In [ ]:
df_new.year.value_counts()

# find out how much bought is total on which day

In [ ]:
# 
daily_sales = df.groupby(["day"])["book_name"].agg(["count"]).reset_index()
daily_sales.sort_values("day",ascending = True)

In [ ]:
plt.figure(figsize = (12,8))
sns.barplot(x =daily_sales["day"], y =daily_sales["count"],color = "Blue",label = "count")
plt.xlabel("Days")
plt.ylabel("item")
plt.title("Sales by Days")
plt.xticks(rotation = 60)
plt.legend()
plt.show()

# find out how much bought is total on which month

In [ ]:
# 
month_sales = df.groupby(["month"])["book_name"].agg(["count"]).reset_index()
month_sales.sort_values("month",ascending = True)

In [ ]:
plt.figure(figsize = (12,8))
sns.barplot(x =month_sales["month"], y =month_sales["count"],color = "Blue",label = "count")
plt.xlabel("month")
plt.ylabel("Total Number of Books Buy")
plt.title("Sales by Month")
plt.xticks(rotation = 60)
plt.legend()
plt.show()

# Sales by month and year

In [ ]:
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')

df1=df_new.groupby(['year']).filter(lambda x: (x['year'] == 2020).any())
df2=df_new.groupby(['year']).filter(lambda x: (x['year'] == 2021).any())
df3=df_new.groupby(['year']).filter(lambda x: (x['year'] == 2019).any())

#Plotting monthly data of number of quantity purchased in 2020 and 2021 
sales_2019=hv.Bars(df3.groupby(['month'])['book_name'].count()).opts(ylabel="# of items", title='# of items sold in 2019')
sales_2020=hv.Bars(df1.groupby(['month'])['book_name'].count()).opts(ylabel="# of items", title='# of items sold in 2020')
sales_2021=hv.Bars(df2.groupby(['month'])['book_name'].count()).opts(ylabel="# of items", title='# of items sold in 2021')

#Merging both plots
(sales_2019 + sales_2020 + sales_2021).opts(opts.Bars(width=380, height=300,tools=['hover'],show_grid=True))

# Week Days Sale

In [ ]:
import altair as alt

#Converting weekday variable to category
temp=df_new.copy()
temp['qty_purchased']=df_new['order_number'].map(df_new['order_number'].value_counts())

#Slicing first 5000 rows as altair library can't plot any data which has record beyond that
temp1=temp[:5000]
temp1.columns
temp1.weekday = temp1.weekday.astype('category') 

#Creating a new dataframe which has the frequency of weekdays
weekday_bin=temp1['weekday'].value_counts().to_frame().reset_index().rename(columns={'index':'weekday','weekday':'count'})

#Plotting bar chart
bars = alt.Chart(weekday_bin).mark_bar(color="darkorange").encode(
    x='weekday',
    y=alt.Y("count",title='Number of purchases')
)

#Adding data labels
text = bars.mark_text(
    align='center',
    baseline='middle',
    dy=-7 ,
    size=15,
).encode(
    text='count',
    tooltip=[alt.Tooltip('weekday'),
            alt.Tooltip('count')]
)

#Combining both
(bars + text).properties(
    width=800,
    height=400,
    title="Number of quantity purchases across weekdays"
)

# Top 10 book sales

In [ ]:
import seaborn as sns

#Setting plot style
plt.figure(figsize = (15, 8))
plt.style.use('seaborn-white')

#Top 10 fast moving products
plt.subplot(1,2,1)
ax=sns.countplot(y="book_name", hue="year", data=df_new, palette="pastel",
              order=df_new.book_name.value_counts().iloc[:10].index)

ax.set_xticklabels(ax.get_xticklabels(),fontsize=11,rotation=40, ha="right")
ax.set_title('Top 10 book sales',fontsize= 22)
ax.set_xlabel('Total # of items purchased',fontsize = 20) 
ax.set_ylabel('Top 10 items', fontsize = 20)
plt.tight_layout()



# 10 Books sale by bottom

In [ ]:
#Bottom 10 fast moving products
plt.subplot(1,2,2)
ax=sns.countplot(y="book_name", hue="year", data=df_new, palette="pastel",
              order=df_new.book_name.value_counts().iloc[-10:].index)
ax.set_xticklabels(ax.get_xticklabels(),fontsize=11,rotation=40, ha="right")
ax.set_title('10 Books sale by bottom',fontsize= 22)
ax.set_xlabel('Total # of books purchased',fontsize = 20) 
ax.set_ylabel('Bottom 10 book', fontsize = 20)
plt.tight_layout()

In [ ]:
#df_new.assign(Book=df_new.book_name.str.split("/")).explode('book_name')

# Which words used mostly 

In [ ]:
from wordcloud import WordCloud,STOPWORDS

#Wordcloud
wordcloud = WordCloud(
    width = 3000,
    height = 2000,
    background_color = 'white').generate("".join(str(df_new['book_name'])))
fig = plt.figure(
    figsize = (50, 30),
    facecolor = 'k',
    edgecolor = 'k')

#Display plot
plt.imshow(wordcloud, interpolation = 'bilinear')
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

# Seperate Book with "/"

In [ ]:
#df_new.assign(book_name_seperate=df_new.book_name.str.split("/")).explode('book_name')

# Top 10 City

In [ ]:
city_sales1 = df_new['billing_city'].value_counts()[:10].index.tolist()
city_sales2 = df_new['billing_city'].value_counts().unique()
city_sales = list(zip(city_sales1, city_sales2)) 
city_sales = pd.DataFrame(city_sales, 
                  columns = ['billig_city', 'counts']) 
city_sales

In [ ]:
plt.figure(figsize = (12,8))
sns.barplot(x =city_sales["billig_city"], y =city_sales["counts"],color = "Blue",label = "count")
plt.xlabel("City")
plt.ylabel("Total numbers of sales")
plt.title("Sales by City")
plt.xticks(rotation = 60)
plt.legend()
plt.show()


In [ ]:
# import plotly.express as px

# fig = px.sunburst(df_new, path=['year', 'month', 'day', 'book_name'],title="Dont Forget to Click Chart to Examine Deeply ")
# fig.show()

In [ ]:
# targets = list(dict(df_new['book_name'].value_counts()).keys())
# values = list(dict(df_new['book_name'].value_counts()).values())

# fig = px.pie(
#     values=values, 
#     names=targets,
#     title='Book Name',
#     color_discrete_sequence=['darkcyan', 'lawngreen']
# )
# fig.show()

In [ ]:
# import squarify

# plt.figure(figsize = (50, 30))
# squarify.plot(sizes = df_new.book_name.value_counts().values, alpha = 0.8,
#               label = df_new.book_name.unique(), text_kwargs={'fontsize':18})
# plt.title('Book Name', fontsize = 30)
# plt.axis('off')
# plt.show()

# Apriori Algorithm

**Now, we need to run apriori algorithm to get insight that if a customer buys one item which item he/she buys next.**

In [ ]:
hot_encoded_df = df_new.groupby(['order_number', 'book_name'])['book_name'].count().unstack().reset_index().fillna(0).set_index('order_number')


In [ ]:
hot_encoded_df.head()

**Above lineAbove line of code is transfrom data to make items as columns and each transaction as a row and count same Items bought in one transaction but fill other cloumns of the row with 0 to represent item which are not bought.**

In [ ]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1
hot_encoded_df = hot_encoded_df.applymap(encode_units)

In [ ]:
frequent_itemsets = apriori(hot_encoded_df, min_support=0.01, use_colnames=True)

* Support is an indication of how frequently the itemset appears in the dataset.
* Confidence is an indication of how often the rule has been found to be true.


In [ ]:
rules = association_rules(frequent_itemsets, metric='lift', min_threshold=1)
rules.head(10)

**We only want to see the rules where confidence is greater than or equal to 50% so:**

In [ ]:
rules[ (rules['lift'] >= 1) & (rules['confidence'] >= 0.5)]

For instance from the last rule we can see that (مشین لرننگ, ARTIFICIAL INTELLIGENCE) and (ڈیٹا سائنس) are commonly bought together. This makes sense since people who purchase (مشین لرننگ, ARTIFICIAL INTELLIGENCE) would like to have (ڈیٹا سائنس) with it.

The support value for the this rule is 0.016109. This number is calculated by dividing the number of transactions containing toast divided by total number of transactions. The confidence level for the rule is 0.04995 which shows that out of all the transactions that contain toast , 76.69% of the transactions also contain (ڈیٹا سائنس). Finally, the lift of 15.34 tells us that (مشین لرننگ, ARTIFICIAL INTELLIGENCE) is 15.34 times more likely to be bought by the customers who buy (مشین لرننگ, ARTIFICIAL INTELLIGENCE) compared to the default likelihood of the sale of (ڈیٹا سائنس).


In [ ]:
support = rules['support'] #rules.to_numpy(columns=['support']) #as_matrix
confidence = rules['confidence'] #rules.to_numpy(columns=['confidence']) 
import seaborn as sns

for i in range (len(support)):
    support[i] = support[i]
    confidence[i] = confidence[i]
    
plt.title('Assonciation Rules')
plt.xlabel('support')
plt.ylabel('confidance')
sns.regplot(x=support, y=confidence, fit_reg=False)

In [ ]:
df_new.head()

In [ ]:
df_new.shape

In [ ]:
df_new.describe()

# What's Next?

* Try to clean city column with fuzzywuzzy
* Predict next hour order
* Predict next day order